In [439]:
!pip install pyinterval

In [440]:
from interval import interval
import pandas as pd

In [441]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_middle_point(interv):
    return (interv[0].sup + interv[0].inf) / 2

def to_string(interv):
    return f"[{interv[0].inf:4.3f}, {interv[0].sup:4.3f}]"

In [442]:
def calculate_volterra1(f, expected, k, y0, a, b, c, d, p):
    h = (b - a) / p
    x = []
    y = []
    exp = []
    exp.append(interval[expected(a),expected(a)])
    for i in range(0, p):
        xi = a + i*h
        xinext = xi + h
        x.append(interval[xi, xinext])
        sum = 0
        for j in range(0, i):
            sum += k(x[i], x[j], y0[j])
        y.append(f(x[i]) + h*sum + interval[0,h]*k(x[i], x[i], y0[i]))
        exp.append(expected(xi))
    u = []
    u.append(y0[0])
    for i in range(0,p-1):
        u.append(y[i]&y[i+1])
    return x, y, u, exp

In [452]:
a = 0
b = 1/4
c = 0
d = 2
p = 10
n = 3
y0 = []
for i in range(p):
    y0.append(interval[c,d])
func = lambda x: 1
k = lambda x,s,y : y**2
kdf = lambda x,s,y : 2*y
expected = lambda x: 1/(1-x)

In [453]:
for iteration in range(n):
    print(f"\nn={iteration+1}")
    x, y, u, exp = calculate_volterra1(func, expected, k, y0, a, b, c, d, p)
    tbl = pd.DataFrame(columns=['xi', 'yi', 'ui', 'm(ui)', 'expected', 'e'])
    for i in range(p):
        mid_point=calculate_middle_point(u[i])
        real = expected(x[i][0].inf)
        tbl.loc[len(tbl)] = [to_string(x[i]),to_string(y[i]),to_string(u[i]),f"{mid_point:4.4}",f"{real:4.4}",abs(mid_point-real)]
    y0 = y
    print(tbl)


n=1
               xi              yi              ui m(ui) expected         e
0  [0.000, 0.025]  [1.000, 1.100]  [0.000, 2.000]   1.0      1.0  0.000000
1  [0.025, 0.050]  [1.000, 1.200]  [1.000, 1.100]  1.05    1.026  0.024359
2  [0.050, 0.075]  [1.000, 1.300]  [1.000, 1.200]   1.1    1.053  0.047368
3  [0.075, 0.100]  [1.000, 1.400]  [1.000, 1.300]  1.15    1.081  0.068919
4  [0.100, 0.125]  [1.000, 1.500]  [1.000, 1.400]   1.2    1.111  0.088889
5  [0.125, 0.150]  [1.000, 1.600]  [1.000, 1.500]  1.25    1.143  0.107143
6  [0.150, 0.175]  [1.000, 1.700]  [1.000, 1.600]   1.3    1.176  0.123529
7  [0.175, 0.200]  [1.000, 1.800]  [1.000, 1.700]  1.35    1.212  0.137879
8  [0.200, 0.225]  [1.000, 1.900]  [1.000, 1.800]   1.4     1.25  0.150000
9  [0.225, 0.250]  [1.000, 2.000]  [1.000, 1.900]  1.45     1.29  0.159677

n=2
               xi              yi              ui  m(ui) expected         e
0  [0.000, 0.025]  [1.000, 1.030]  [1.000, 1.100]   1.05      1.0  0.050000
1  [0.025, 0.